In [373]:
from pathlib import Path
from procureme.vectordb.lance_vectordb import LanceDBVectorStore
from openai import OpenAI
from dotenv import load_dotenv
import os
from procureme.clients.ollama_embedder import OllamaEmbeddingClient
import json
from enum import StrEnum
import logging
from typing import Optional

In [374]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("AppAgent")

In [375]:
class AiModels(StrEnum):
    GEMMA3 = "gemma3:4b"
    GPT3 = "gpt-3.5-turbo-0125"
    GPT4_1_NANO = "gpt-4.1-nano"
    GPT4_1_MINI = "gpt-4.1-mini-2025-04-14"
    NOMIC = "nomic-embed-text:v1.5"
    LLAMA31 = "llama3.1:latest"

In [376]:
load_dotenv("../.env", override=True)

True

In [377]:
oai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [378]:
ollama_client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama'
)

In [379]:
test_message = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what do you know about Retrival Augmented Generation RAG? tell me in 1 sentence"},
    ]

In [380]:
response = ollama_client.chat.completions.create(
  model=AiModels.GEMMA3,
  messages=test_message
)
print(response.choices[0].message.content)

2025-06-01 23:59:26,957 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


Retrieval-Augmented Generation (RAG) combines the strengths of large language models with external knowledge retrieval systems to generate more accurate, contextually relevant, and informed responses.


In [381]:
response = oai_client.chat.completions.create(
  model=AiModels.GPT3,
  messages=test_message
)
print(response.choices[0].message.content)

2025-06-01 23:59:27,595 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RAG (Retrieval-Augmented Generation) is a natural language processing model that combines a retriever and a generator to improve text generation by first retrieving relevant information and then generating responses based on that information.


In [382]:
def get_vector_store():
    db_path = Path().absolute().parent.joinpath("vectordb", "contracts.db")
    vector_table = "contracts_naive"
    emb_client = OllamaEmbeddingClient()
    vector_store = LanceDBVectorStore(db_path=db_path, table_name=vector_table, embedding_client=emb_client)
    logger.info("Vector DB Loaded successfully.")
    return vector_store

In [383]:
VECTOR_STORE = get_vector_store()

2025-06-01 23:59:27,783 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-06-01 23:59:27,787 - AppAgent - INFO - Vector DB Loaded successfully.


In [399]:
query= "What do you know about Supplier: Alpha Suppliers Inc. and what it supplies?"
VECTOR_STORE.search(query=query, top_k=5)[1]

2025-06-02 00:06:54,168 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"


{'chunk_id': 'CW0303.pdf-part - 4',
 'doc_id': 'c95b4916-7170-422e-8371-1d7afa17fc25',
 'file_name': 'CW0303.pdf',
 'total_pages': 4,
 'content': '................................................................. (Supplier legal entity name) that are in effect \n(“Agreement”). \nWith the signing of this document, we intend to amend the Agreement in order to fully \nincorporate the above Environmental Sustainability Criteria. \n................................................................. (Supplier legal entity name) at \n........................................................................................... (Supplier address), acknowledges the \nre-ceipt of the Plasma Corporation’ Environmental Sustainability Criteria and agrees to be bound \nby them. \nThis document shall be deemed part of the Agreement and any future reference to the Agreement \nshall include the Environmental Sustainability Criteria. \n \n \n[Signatures] \n________________________ \nJohn Doe \nProcurement Ma

In [385]:
def chat(messages, model=AiModels.GPT3, temperature=0, config={}):
    response = oai_client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages,
        **config,
    )
    logger.info(f"Chat endpoint called with Model {model}, Request: {messages[-1]}")
    content = response.choices[0].message.content
    return content

In [386]:
def tool_choice(messages, model=AiModels.GPT4_1_MINI, temperature=0, tools=[], config={}):
    response = oai_client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages,
        tools=tools or None,
        **config,
    )
    tools = response.choices[0].message.tool_calls
    return tools

In [387]:
query_update_prompt = """
    You are an expert at updating questions to make them ask for one thing only, more atomic, specific and easier to find the answer for.
    You do this by filling in missing information in the question, with the extra information provided to you in previous answers. 
    
    You respond with the updated question that has all information in it.
    Only edit the question if needed. If the original question already is atomic, specific and easy to answer, you keep the original.
    Do not ask for more information than the original question. Only rephrase the question to make it more complete.
    
    JSON template to use:
    {
        "question": "question1"
    }
"""

def query_update(input: str, answers: list[any]) -> str:
    logger.info("=== Entering Query Update Node ===")
    logger.info(f"Query Update endpoint called with Intermediate User and Assistant turns: {len(answers)}")

    messages = [
        {"role": "system", "content": query_update_prompt},
        *answers,
        {"role": "user", "content": f"The user question to rewrite: '{input}'"},
    ]

    config = {"response_format": {"type": "json_object"}}
    output = chat(messages, model = AiModels.GPT4_1_MINI, config=config, )
    try:
        updated_question = json.loads(output)["question"]
        logger.info(f"Updated Query: {updated_question}")
        return updated_question
    except json.JSONDecodeError:
        print("Error decoding JSON")
    return []

In [388]:
# query_update("What do you know about Supplier: Alpha Suppliers Inc. and what they usually supply?", answers=[])

# Tools and Their Definitions

In [389]:
answer_given_description = {
    "type": "function",
    "function": {
        "name": "answer_given",
        "description": "If the conversation already contains a complete answer to the question, "
        "use this tool to extract it. Additionally, if the user engages in small talk, "
        "use this tool to remind them that you can only answer questions about movies and their cast.",
        "parameters": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "description": "Respond directly with the answer",
                }
            },
            "required": ["answer"],
        },
    },
}


def answer_given(answer: str):
    """Extract the answer from a given text."""
    logger.info("Answer found in text: %s", answer)
    return answer


In [390]:
retriver_tool_description = {
    "type": "function",
    "function": {
        "name": "retriver_tool",
        "description": "Query the vector database with a user question to pull the most relevant chunks. When other tools don't fit, fallback to use this one.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The user question or query to find the answer for",
                }
            },
            "required": ["question"],
        },
    },
}


def retriver_tool(question: str):
    """Query the database with a user question."""
    logger.info("=== Entering Retrival Tool ===")
    try:
        vector_store = get_vector_store()
        documents = vector_store.search(query=question, top_k=5)
        context = "\n\n".join([doc["content"] for doc in documents])
        logger.info("Retrival Tool executed successfully.")
        return context
    except Exception as e:
        return [f"Could not query vector store, cause an error: {e}"]

In [391]:
tools = {
    "retriver_tool": {
        "description": retriver_tool_description,
        "function": retriver_tool
    },
    "answer_given": {
        "description": answer_given_description,
        "function": answer_given
    }
}

In [ ]:
tool_picker_prompt = """
    Your job is to chose the right tool needed to respond to the user question. 
    The available tools are provided to you in the prompt.
    Make sure to pass the right and the complete arguments to the chosen tool.
"""

def handle_tool_calls(tools: dict[str, any], llm_tool_calls: list[dict[str, any]]):
    logger.info("=== Selecting Tools ===")
    output = []
    available_tools = [tool["description"]["function"]["name"] for tool in tools.values()]
    if llm_tool_calls:
        tool_list = [tool_call.function.name for tool_call in llm_tool_calls]
        logger.info(f"Follwing tools are select for execution: {tool_list} from available tools: {available_tools}")
        for tool_call in llm_tool_calls:
            function_to_call = tools[tool_call.function.name]["function"]
            function_args = json.loads(tool_call.function.arguments)
            res = function_to_call(**function_args)
            output.append(res)
    logger.info(f"Tool execution finished!")
    return output

In [393]:
def route_question(question: str, tools: dict[str, any], answers: list[dict[str, str]]):
    logger.info("=== Entering Tool Selector Router ===")
    llm_tool_calls = tool_choice(
        [
            {
                "role": "system",
                "content": tool_picker_prompt,
            },
            *answers,
            {
                "role": "user",
                "content": f"The user question to find a tool to answer: '{question}'",
            },
        ],
        model = AiModels.GPT4_1_MINI,
        tools=[tool["description"] for tool in tools.values()],
    )
    return handle_tool_calls(tools, llm_tool_calls)

def handle_user_input(input: str, answers: Optional[list[dict[str, str]]] = None):  
    answers = answers if answers else []
    logger.info(f"User input: {input}, with initial No. of User and Assistant turns: {len(answers)}")
    updated_question = query_update(input, answers)

    response  = route_question(updated_question, tools, answers)
    answers.append({"role": "assistant", "content": f"For the question: '{updated_question}', we have the answer: '{json.dumps(response)}'"})
    return answers

In [397]:
answer_critique_prompt = """
    You are an expert at identifying if questions has been fully answered or if there is an opportunity to enrich the answer.
    The user will provide a question, and you will scan through the provided information to see if the question is answered.
    If anything is missing from the answer, you will provide a set of new questions that can be asked to gather the missing information.
    All new questions must be complete, atomic and specific.
    However, if the provided information is enough to answer the original question, you will respond with an empty list.

    JSON template to use for finding missing information:
    {
        "questions": ["question1", "question2"]
    }
"""

def critique_answers(question: str, answers: list[dict[str, str]]) -> list[str]:
    messages = [
        {
            "role": "system",
            "content": answer_critique_prompt,
        },
        *answers,
        {
            "role": "user",
            "content": f"The original user question to answer: {question}",
        },
    ]
    config = {"response_format": {"type": "json_object"}}
    output = chat(messages, model=AiModels.GPT3, config=config)
    logger.info(f"Answer critique response: {output}")
    try:
        return json.loads(output)["questions"]
    except json.JSONDecodeError:
        print("Error decoding JSON")
    return []

In [395]:
main_prompt = """
    Your job is to help the user with their questions.
    You will receive user questions and information needed to answer the questions
    If the information is missing to answer part of or the whole question, you will say that the information 
    is missing. You will only use the information provided to you in the prompt to answer the questions.
    You are not allowed to make anything up or use external information.
"""

def main(input: str):
    answers = handle_user_input(input)
    critique = critique_answers(input, answers)

    if critique:
        answers = handle_user_input(" ".join(critique), answers)

    llm_response = chat(
        [
            {"role": "system", "content": main_prompt},
            *answers,
            {"role": "user", "content": f"The user question to answer: {input}"},
        ],
        model=AiModels.GPT4_1_MINI,
    )

    return llm_response

In [234]:
# response = main("Who's the main actor in the movie Matrix and what other movies is that person in?")
# print(f"Main response: {response}")

In [398]:
response = main("What do you know about Supplier: Alpha Suppliers Inc. and what it supplies?")
print(f"Main response: {response}")

2025-06-02 00:00:22,511 - AppAgent - INFO - User input: What do you know about Supplier: Alpha Suppliers Inc. and what it supplies?, with initial No. of User and Assistant turns: 0
2025-06-02 00:00:22,513 - AppAgent - INFO - === Entering Query Update Node ===
2025-06-02 00:00:22,514 - AppAgent - INFO - Query Update endpoint called with Intermediate User and Assistant turns: 0
2025-06-02 00:00:23,669 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-02 00:00:23,672 - AppAgent - INFO - Chat endpoint called with Model gpt-4.1-mini-2025-04-14, Request: {'role': 'user', 'content': "The user question to rewrite: 'What do you know about Supplier: Alpha Suppliers Inc. and what it supplies?'"}
2025-06-02 00:00:23,674 - AppAgent - INFO - Updated Query: What products or services does Alpha Suppliers Inc. provide as a supplier?
2025-06-02 00:00:23,675 - AppAgent - INFO - === Entering Tool Selector Router ===
2025-06-02 00:00:24,529 - httpx - I

Main response: Alpha Suppliers Inc. is a supplier engaged under a Procurement Contract with XYZ Corporation. According to the contract details, Alpha Suppliers Inc. provides office stationery and supplies, including items such as pens, paper, folders, and other stationery items as specified in the attached Purchase Order. Additionally, they supply any additional stationery items requested by the Buyer during the contract period.


# Pulling Full Document